In [ ]:
import pandas as pd
import numpy as np
import gensim.models
import seaborn as sb
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow==1.13.1
! pip install tensorflow-hub==0.7.0


     |████████████████████████████████| 92.6MB 41kB/s 
     |████████████████████████████████| 3.2MB 40.8MB/s 
     |████████████████████████████████| 368kB 51.3MB/s 
     |████████████████████████████████| 51kB 5.0MB/s 
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 92kB 3.5MB/s 
  Found existing installation: tensorflow-hub 0.11.0
    Uninstalling tensorflow-hub-0.11.0:
      Successfully uninstalled tensorflow-hub-0.11.0


In [ ]:
import tensorflow_hub as hub
import tensorflow as tf
class UseSentenceEmbedding():
    def __init__(self):
        # g = tf.Graph()
        with tf.device('/GPU:0'):
        # We will be feeding 1D tensors of text into the graph.
            self.text_input = tf.placeholder(dtype=tf.string, shape=[None])
            
            #kindly replace the location in hub.module with the url commented out below

            # "https://tfhub.dev/google/universal-sentence-encoder-large/3"
            embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-large/3")
            self.embedded_text = embed(self.text_input)
            init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
        # g.finalize()

        self.session = tf.Session(config=tf.ConfigProto( allow_soft_placement=True))
        self.session.run(init_op)
        print("init _____")



    def get_tokenized_sents_embeddings_USE(self, sents,expand=False):

           
        vectors_USE =  self.session.run(self.embedded_text, feed_dict={self.text_input: sents})

        return vectors_USE

use_embedding = UseSentenceEmbedding()

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


init _____


In [ ]:
!cp "/content/drive/MyDrive/Information_retrieval_project/khan_acad/train_khan_acad.csv" /content
!cp "/content/drive/MyDrive/Information_retrieval_project/khan_acad/test_khan_acad.csv" /content
!cp "/content/drive/MyDrive/Information_retrieval_project/khan_acad/val_khan_acad.csv" /content


In [ ]:
import pandas as pd
train_data = pd.read_csv("train_khan_acad.csv")
test_data = pd.read_csv("test_khan_acad.csv")
val_data = pd.read_csv("val_khan_acad.csv")
train_data
from google.colab import files

In [ ]:
import re
def clean_sentence(question):
  # print(question)
  question = re.sub('<[^>]*>', ' ',question)
  question = re.sub(' +', ' ', question)
  question = re.sub('\xa0','',question)
  question = question.rstrip()
  question = re.sub('nan','',question)
  question = re.sub(u'\u2004','',question)
  question = re.sub(u'\u2009','',question)

  # question = question.decode("utf-8")
  # question = question.replace(u'\u200\d*','').encode("utf-8")
  question = re.sub('&nbsp','',question)
  question = re.sub('&ndash','',question)
  question = re.sub('\r','',question)
  question = re.sub('\t','',question)
  question = re.sub('\n',' ',question)

  question = re.sub('MathType@.*','',question)
  question = re.sub('&thinsp','',question)
  question = re.sub('&times','',question)
  question = re.sub('\u200b','',question)
  question = re.sub('&rarr;;;','',question)

  return question

In [ ]:

from sklearn.preprocessing import LabelEncoder

LE = LabelEncoder()
LE.fit_transform(pd.concat([train_data['hierarchy'],test_data['hierarchy']]))
train_data['label'] = LE.transform(train_data['hierarchy'])
train_data.head()

,video_transcripts,hierarchy,label
0,In the last couple of videos we saw that we c...,math>>multivariable-calculus>>multivariable-de...,354
1,- What we're going to do in this video is gi...,science>>ap-biology>>natural-selection,422
2,"So once again, we have three equal, or we say...",math>>pre-algebra>>pre-algebra-equations-expre...,384
3,- Liz's math test included a survey question...,math>>engageny-alg-1>>alg1-2,231
4,- The following two equations form a linear s...,math>>algebra-home>>alg-system-of-equations,99


In [ ]:
def get_labels(prediction):
    predicted_label =  LE.inverse_transform([prediction])
    return predicted_label[0]

In [ ]:
# LE_test = LabelEncoder()

test_data['label'] = LE.transform(test_data['hierarchy'])
test_data.head()

,video_transcripts,hierarchy,label
0,- What I hope to do in this video is get fam...,math>>math1>>x89d82521517266d4:functions,335
1,In the last video we were able to set up this...,math>>old-ap-calculus-ab>>ab-applications-defi...,357
2,- In previous videos we talk about GDP as th...,economics-finance-domain>>ap-macroeconomics>>e...,3
3,- So what we're gonna do in this video is se...,math>>old-integral-calculus>>definite-integral...,378
4,- So I've said that if you have a vector fie...,math>>multivariable-calculus>>multivariable-de...,354


In [ ]:
val_data['label'] = LE.transform(val_data['hierarchy'])
val_data.head()

,video_transcripts,hierarchy,label
0,Find the probability of rolling doubles on tw...,math>>precalculus>>x9e81a4f98389efdf:prob-comb,395
1,"After the food is swallowed, it leaves the m...",science>>health-and-medicine>>human-anatomy-an...,497
2,Let's now talk about what is easily one of th...,math>>geometry>>hs-geo-trig,256
3,The goal in this video is to essentially prov...,science>>chemistry>>thermodynamics-chemistry,472
4,"A line goes through the points (-1, 6) and (5...",math>>in-in-grade-11-ncert>>in-in-class11-stra...,304


In [ ]:
train_features, test_features, train_labels, test_labels = train_data["video_transcripts"].values,test_data["video_transcripts"].values,train_data["label"].values,test_data["label"].values
val_features,val_labels = val_data["video_transcripts"].values, val_data["label"].values

In [ ]:
train_features = use_embedding.get_tokenized_sents_embeddings_USE(train_features)
test_features = use_embedding.get_tokenized_sents_embeddings_USE(test_features)
print(train_features.shape,test_features.shape)

(4188, 512) (1047, 512)


In [62]:
import joblib
joblib.dump(train_features,'train_features_log_khan_acad')

['train_features_log_khan_acad']

In [63]:
joblib.dump(test_features,'test_features_log_khan_acad')

['test_features_log_khan_acad']

In [64]:
!mv test_features_log_khan_acad "/content/drive/MyDrive/DL_project/"
!mv train_features_log_khan_acad "/content/drive/MyDrive/DL_project/"

In [65]:
!mv  "/content/drive/MyDrive/DL_project/test_features_log_khan_acad" "/content/drive/MyDrive/Information_retrieval_project/"
!mv  "/content/drive/MyDrive/DL_project/train_features_log_khan_acad" "/content/drive/MyDrive/Information_retrieval_project/"

In [ ]:
#parameters = {'solver':['newton-cg', 'lbfgs','sag', 'saga'], 'C':[0.01,0.1,1, 10]}
log = LogisticRegression(max_iter=1000)
#clf = GridSearchCV(log, parameters)
log.fit(train_features, train_labels)
#clf.best_params_

In [20]:
y_predict = log.predict_proba(test_features)
print(y_predict.shape)

(1047, 569)
(1047, 569)
(1047, 569)


In [35]:
get_labels(0)

'economics-finance-domain>>ap-macroeconomics>>ap-financial-sector'

In [57]:
final_predictions =[]
flat_predictions = y_predict
for prediction in flat_predictions:
  final_predictions.append(np.argsort(prediction.reshape(1,-1), axis=1)[:,-20:].squeeze())

In [58]:
final_predictions[1]

array([137, 349, 173, 365, 222, 543,  96,  94, 179, 239, 174, 352, 100,
       233, 360,  93, 350, 361,  87, 351])

In [47]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(test_labels, log.predict(test_features), average='weighted', zero_division=0)

(0.19197707736389685, 0.19197707736389685, 0.19197707736389685, None)

In [50]:
import tensorflow as tf
y_true = np.array(test_labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 5)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 5)

tmp_rank = tf.nn.top_k(y_pred, 5)
stream_vars = [i for i in tf.local_variables()]

with tf.Session(config=tf.ConfigProto( allow_soft_placement=True)) as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(y_pred))

(1047, 5) (1047,)
update_recall:  0.11174785100286533
recall 0.11174785100286533
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 117.0, 930.0, 0.0, 0.0]
TMP_RANK:  [[173  93 233 476 327]
 [ 93 350 361  87 351]
 [  8  14   3   0  17]
 ...
 [350 320 327 321 352]
 [327 241 403 148 392]
 [493 495 499 417 494]]


In [53]:
import tensorflow as tf
y_true = np.array(test_labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 10)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 10)

tmp_rank = tf.nn.top_k(y_pred, 10)
stream_vars = [i for i in tf.local_variables()]

with tf.Session(config=tf.ConfigProto( allow_soft_placement=True)) as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(y_pred))

(1047, 10) (1047,)
update_recall:  0.1766953199617956
recall 0.1766953199617956
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 185.0, 862.0, 0.0, 0.0]
TMP_RANK:  [[565 239  96 ... 233 476 327]
 [174 352 100 ... 361  87 351]
 [ 13  15  10 ...   3   0  17]
 ...
 [550 309 351 ... 327 321 352]
 [407 495 234 ... 403 148 392]
 [542 451 486 ... 499 417 494]]


In [56]:
import tensorflow as tf
y_true = np.array(test_labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 15)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 15)

tmp_rank = tf.nn.top_k(y_pred, 15)
stream_vars = [i for i in tf.local_variables()]

with tf.Session(config=tf.ConfigProto( allow_soft_placement=True)) as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(y_pred))

(1047, 15) (1047,)
update_recall:  0.21967526265520534
recall 0.21967526265520534
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 230.0, 817.0, 0.0, 0.0]
TMP_RANK:  [[109 352 349 ... 233 476 327]
 [543  96  94 ... 361  87 351]
 [ 18  21   9 ...   3   0  17]
 ...
 [476 489 543 ... 327 321 352]
 [160 152 380 ... 403 148 392]
 [543 473 413 ... 499 417 494]]


In [59]:
import tensorflow as tf
y_true = np.array(test_labels)
y_true = tf.identity(y_true)
y_pred = np.array(final_predictions)
y_pred = tf.identity(y_pred)
print(y_pred.shape,y_true.shape)
k = 8
recall, update_recall = tf.compat.v1.metrics.recall_at_top_k(y_true, y_pred, 20)
precision, update_precision = tf.compat.v1.metrics.precision_at_top_k(y_true, y_pred, 20)

tmp_rank = tf.nn.top_k(y_pred, 20)
stream_vars = [i for i in tf.local_variables()]

with tf.Session(config=tf.ConfigProto( allow_soft_placement=True)) as sess:
    sess.run(tf.local_variables_initializer())
    # print("precision",sess.run(update_precision))
    # print("precision",sess.run(precision))

    print("update_recall: ",sess.run(update_recall ))
    print("recall",sess.run(recall))

    print("STREAM_VARS: ",(sess.run(stream_vars)))
    print("TMP_RANK: ",sess.run(y_pred))

(1047, 20) (1047,)
update_recall:  0.2588347659980898
recall 0.2588347659980898
STREAM_VARS:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 271.0, 776.0, 0.0, 0.0]
TMP_RANK:  [[232 380 169 ... 233 476 327]
 [137 349 173 ... 361  87 351]
 [233  20 416 ...   3   0  17]
 ...
 [353 403 254 ... 327 321 352]
 [ 84 201 488 ... 403 148 392]
 [539 502 472 ... 499 417 494]]
